# Word Document to Graph
- using neo4j knowledge graph


In [2]:
!pip install Spire.Doc


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/27.0 MB ? eta -:--:--
    --------------------------------------- 0.3/27.0 MB 10.2 MB/s eta 0:00:03
   - -------------------------------------- 0.9/27.0 MB 11.2 MB/s eta 0:00:03
   -- ------------------------------------- 1.4/27.0 MB 11.3 MB/s eta 0:00:03
   -- ------------------------------------- 2.0/27.0 MB 11.4 MB/s eta 0:00:03
   --- ------------------------------------ 2.5/27.0 MB 11.5 MB/s eta 0:00:03
   ---- ----------------------------------- 3.1/27.0 MB 12.2 MB/s eta 0:00:02
   ----- ---------------------------------- 3.6/27.0 MB 12.0 MB/s eta 0:00:02
   ------ --------------------------------- 4.1/27.0 MB 12.0 MB/s eta 0:00:02
   ------ --------------------------------- 4.7/27.0 MB 12.0 MB/s eta 0:00:02
   ------- -------------------------------- 5.2/27.0 MB 11.9 MB/s eta 0:00:02
   -------- ------------------------------- 5.8/27.0 MB 11.9 MB/s eta 0:00:02
   -------- ------------------------------- 6.0/27.0 MB 11.3 MB/s eta 0

In [2]:
# create function to extract text from a folder of .docx files
import os
from spire.doc import Document

# Function Definition
def extract_text_from_folder(folder_path):
    """
    Extracts text from all .docx files in a folder and stores them in a list.
    Removes first line of text from each document to remove warning or metadata lines
    from Spire.Doc that is not part of the actual document content.

    Args:
    folder_path (str): The path to the folder containing the .docx files.

    Returns:
    list: A list containing the extracted text from each document.
    """
    documents_text = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".docx"):
            filepath = os.path.join(folder_path, filename)
        try:
            document = Document()
            document.LoadFromFile(filepath)
            # Extract text and remove the first line (if it exists)
            text = document.GetText()
            lines = text.splitlines(keepends=True)  # Split by lines, keeping newlines
            if lines:  # Check if there are any lines
                text_without_first_line = "".join(lines[1:])  # Join lines from index 1 onwards
            else:
                text_without_first_line = ""  # Empty string if no lines

            documents_text.append(text_without_first_line)

        except Exception as e:
            print(f"Error processing file {filename}: {e}")
    return documents_text

In [3]:
# Call function on "Documents" folder
folder_path = r"C:\Users\Z4N3R\OneDrive\Desktop\Documents\GitHub\RAGTAGbot\documents"
documents_text = extract_text_from_folder(folder_path)

In [7]:
print(documents_text[0])

Chee Kai Ming World City Prize 2020 
Ecodora Trip Report – Summary

Background:

Ecodora, a flourishing city dedicated to sustainability, harnesses renewable energy sources, boasts a recycling rate of 75%, and has implemented an efficient public transportation system that has reduced car usage by 40%. Verdant parks and green spaces encompass 25% of the city, providing a tranquil oasis for residents. Ecodora's education system prioritizes environmental stewardship and critical thinking, with 90% of high school graduates pursuing higher education. The city actively engages in global collaborations, sharing innovative ideas and supporting sustainable initiatives worldwide. Residents of Ecodora enjoy an exceptional quality of life, with an impressive 85% satisfaction rating. Ecodora exemplifies how sustainability can create a harmonious and prosperous urban environment.


Key Strategies Adopted:
1. Eco-Transit Revolution:
* Strategy: Ecodora has prioritized public transportation, investing

### Generate Main Body Text
- will be done for each document
- invoking llm

pw = "dj-FX177bkCtDUcAqKXY9obqxkbhU-5SQ7ZoENa3GT0"
uri = "neo4j+s://e143a661.databases.neo4j.io"
uname = "neo4j"

In [5]:
!pip install langchain-openai

   ---------------------------------------- 0.0/45.9 kB ? eta -:--:--
   -------- ------------------------------- 10.2/45.9 kB ? eta -:--:--
   -------- ------------------------------- 10.2/45.9 kB ? eta -:--:--
   -------- ------------------------------- 10.2/45.9 kB ? eta -:--:--
   -------- ------------------------------- 10.2/45.9 kB ? eta -:--:--
   ---------------------------------------- 45.9/45.9 kB 189.7 kB/s eta 0:00:00
   ---------------------------------------- 0.0/327.5 kB ? eta -:--:--
   - -------------------------------------- 10.2/327.5 kB ? eta -:--:--
   - -------------------------------------- 10.2/327.5 kB ? eta -:--:--
   - -------------------------------------- 10.2/327.5 kB ? eta -:--:--
   ---- ---------------------------------- 41.0/327.5 kB 196.9 kB/s eta 0:00:02
   ------- ------------------------------- 61.4/327.5 kB 273.8 kB/s eta 0:00:01
   ------- ------------------------------- 61.4/327.5 kB 273.8 kB/s eta 0:00:01
   ------- ----------------------------


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
from dotenv import load_dotenv
from langchain_community.graphs import Neo4jGraph
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
neo4j_uri = os.getenv("NEO4J_URI")
neo4j_username = os.getenv("NEO4J_USERNAME")
neo4j_password = os.getenv("NEO4J_PASSWORD")

os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["NEO4J_URI"] = neo4j_uri
os.environ["NEO4J_USERNAME"] = neo4j_username
os.environ["NEO4J_PASSWORD"] = neo4j_password
    
graph = Neo4jGraph()

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

llm_transformer = LLMGraphTransformer(llm=llm)

In [8]:
from langchain_text_splitters import TokenTextSplitter
import re

text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
test_text = text_splitter.split_text(documents_text[0])

# clean the text, remove \n and \r
def clean_text(text):
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"\r", " ", text)
    return text

# Call the clean_text function on each text chunk
cleaned_text = [clean_text(chunk) for chunk in test_text]


In [9]:

prompt = f"""
    You will be given text extracted from trip reports from Word documents.
    The text will contain the main body content, as well as irrelevant section's random metadata. 
    Your task is to extract only the relevant body text from the reports, together with its document title and reporting officer, excluding all other irrelevant information.
    To guide you, the reports are typically:
    - written in paragraph form, with 7 chapters regarding the trip and the city visited
    - includes the document title, city & year visited at the beginning of the report and reporting officer at the end
    - includes names of politicians and initiatives organised by the local government
    Please output relevant body text with the corresponding city, year visited and officer, without any additional formatting or comments.

    Extracted document Text:
    {cleaned_text}
    """

In [10]:
# Invoke the LLM model to extract the relevant text
response = llm.invoke(prompt)

In [17]:
response.content

'Document Title: Chee Kai Ming World City Prize 2020\nCity Visited: Ecodora\nYear Visited: 2020\n\nBackground:\nEcodora, a flourishing city dedicated to sustainability, harnesses renewable energy sources, boasts a recycling rate of 75%, and has implemented an efficient public transportation system that has reduced car usage by 40%. Verdant parks and green spaces encompass 25% of the city, providing a tranquil oasis for residents. Ecodora\'s education system prioritizes environmental stewardship and critical thinking, with 90% of high school graduates pursuing higher education. The city actively engages in global collaborations, sharing innovative ideas and supporting sustainable initiatives worldwide. Residents of Ecodora enjoy an exceptional quality of life, with an impressive 85% satisfaction rating. Ecodora exemplifies how sustainability can create a harmonious and prosperous urban environment.\n\nKey Strategies Adopted:\n1. Eco-Transit Revolution:\n* Strategy: Ecodora has prioritiz

In [16]:
from langchain_core.documents import Document

documents = [Document(page_content=response.content)]
# Extract graph data
graph_documents = llm_transformer.convert_to_graph_documents(documents)
# Store to neo4j
graph.add_graph_documents(
  graph_documents, 
  baseEntityLabel=True, 
  include_source=True
)

test import to neo4j graph seems to work nicely, reflected on neo4j workspace as well

In [ ]:
def generate_body_text(text, llm):
    prompt = f"""
    You will be given text extracted from trip reports from Word documents.
    The text will contain the main body content, as well as irrelevant section's random metadata. 
    Your task is to extract only the relevant body text from the reports, together with its document title and reporting officer, excluding all other irrelevant information.
    To guide you, the reports are typically:
    - Is written in paragraph form, with multiple sentences forming a coherent narrative
    - Does not contain short, fragmented phrases or single-sentence headlines
    - Does not include the news outlet name, slogans, or metadata
    - May include quotes or attributions to sources within the paragraphs
    Please output only the extracted body text, without any additional formatting or comments.

    Extracted PDF Text:
    {text}
    """
    response = llm.invoke(prompt)
    
    print(response.content)
    
    clean_text = re.sub(r'\n', '', response.content)
    return clean_text